In [1]:
import os
import shutil
import random

# Define source and destination paths
dataset_path=r"C:\Users\Administrator\Desktop\DRD\archive (16)\gaussian_filtered_images\gaussian_filtered_images"
trian_path=r"C:\Users\Administrator\Desktop\DRD\train"
val_path=r"C:\Users\Administrator\Desktop\DRD\val"

categories=["No_DR" ,"Mild" ,"Moderate" ,"Severe" ,"Proliferate_DR"]

# Split ratio
split_ratio=0.8    # 80% tarin, 20% test

# create trian and val directories
for category in categories:
    os.makedirs(os.path.join(trian_path,category),exist_ok=True)
    os.makedirs(os.path.join(val_path,category),exist_ok=True)

    # Get all images in category folder
    images=os.listdir(os.path.join(dataset_path,category))
    random.shuffle(images)

    # Splitdata
    train_size=int(len(images)* split_ratio)
    train_images=images[:train_size]
    val_images=images[train_size:]

    # Move images to train and val folders
    for img in train_images:
        shutil.move(os.path.join(dataset_path,category,img),os.path.join(trian_path,category,img))

    for img in val_images:
        shutil.move(os.path.join(dataset_path,category,img),os.path.join(val_path,category,img))

print("Dataset succesfully split into train and validation sets✅!")

Dataset succesfully split into train and validation sets✅!


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator



train_dir = r"C:\Users\Administrator\Desktop\DRD\train"
val_dir = r"C:\Users\Administrator\Desktop\DRD\val"

# Define the correct class order
class_mapping = {"No_DR": 0, "Mild": 1, "Moderate": 2, "Severe": 3, "Proliferate_DR": 4}

# Load dataset with the correct class order
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="categorical",  # Multi-class classification
    classes=class_mapping  # ✅ Manually specify class indices
)

val_generator = val_datagen.flow_from_directory(
    val_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="categorical",
    classes=class_mapping  # ✅ Apply same mapping to validation data
)

# Print class indices to verify
print("Class Indices:", train_generator.class_indices)


# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(5, activation="softmax")  # ✅ 5 classes, softmax activation for multi-class
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])  # ✅ Correct loss function

# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save the trained model
model.save(r"C:\Users\Administrator\Desktop\DRD\dr.h5")
print("✅ Model training complete and saved!")

Found 2929 images belonging to 5 classes.
Found 733 images belonging to 5 classes.
Class Indices: {'No_DR': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate_DR': 4}


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 88s 940ms/step - accuracy: 0.4897 - loss: 1.8667 - val_accuracy: 0.6917 - val_loss: 0.8718
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 914ms/step - accuracy: 0.6751 - loss: 0.8978 - val_accuracy: 0.6930 - val_loss: 0.8188
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 913ms/step - accuracy: 0.7163 - loss: 0.8013 - val_accuracy: 0.6903 - val_loss: 0.8050
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 920ms/step - accuracy: 0.6965 - loss: 0.8372 - val_accuracy: 0.6985 - val_loss: 0.8027
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 911ms/step - accuracy: 0.6847 - loss: 0.8509 - val_accuracy: 0.7149 - val_loss: 0.8056
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 85s 921ms/step - accuracy: 0.6945 - loss: 0.8259 - val_accuracy: 0.7067 - val_loss: 0.8223
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 912ms/step - accuracy: 0.7032 - loss: 0.8082 - val_accuracy: 0.7231 - val_loss: 0.7700
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 84s 912ms/step - accuracy: 0.7096 - loss: 0.7843 - val_accu

✅ Model training complete and saved!


In [3]:
train_generator.class_indices

{'No_DR': 0, 'Mild': 1, 'Moderate': 2, 'Severe': 3, 'Proliferate_DR': 4}

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model_path = r"C:\Users\Administrator\Desktop\DRD\dr.h5"
model = tf.keras.models.load_model(model_path)

# Class index mapping as per your custom label order
class_labels = {
    0: "No_DR",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    4: "Proliferate_DR"
}

# Load and preprocess image
img_path = r"C:\Users\Administrator\Desktop\DRD\val\Mild\4e0656629d02.png"  # Change to your image path
img = image.load_img(img_path, target_size=(224, 224))  # Use target_size that matches model input
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]

# Output result
print(f"Prediction: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Prediction: No_DR
